<a href="https://colab.research.google.com/github/snvssk/data298A/blob/development/SolarEnergy/MLModels/XgBoost1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import r2_score, mean_squared_error, make_scorer
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Dataset Import

In [4]:
#df = pd.read_csv('./all_cities_transformed_combined.csv')
df=pd.read_csv('/content/drive/Shareddrives/DATA298/WeatherData/FinalWeatherData/all_cities_transformed_combined.csv')

In [5]:
df.head()

,Unnamed: 0,Year,Month,Day,Hour,Minute,Temperature,Cloud_Type,Dew_Point,GHI,Relative_Humidity,Solar_Zenith_Angle,Surface_Albedo,Pressure,Precipitable_Water,Wind_Direction,Wind_Speed,Location
0,0,2018.0,1.0,1.0,0.0,0.0,11.2,4.0,10.2,0.0,93.63,165.41,0.1,1024.0,1.7,294.0,0.7,117987.0
1,1,2018.0,1.0,1.0,0.0,30.0,11.2,4.0,10.2,0.0,93.63,165.11,0.1,1024.0,1.7,307.0,0.7,117987.0
2,2,2018.0,1.0,1.0,1.0,0.0,11.1,4.0,10.1,0.0,93.38,162.29,0.1,1024.0,1.7,321.0,0.7,117987.0
3,3,2018.0,1.0,1.0,1.0,30.0,11.0,0.0,10.1,0.0,94.00,157.91,0.1,1024.0,1.8,332.0,0.7,117987.0
4,4,2018.0,1.0,1.0,2.0,0.0,10.9,0.0,9.9,0.0,93.32,152.72,0.1,1024.0,1.8,344.0,0.7,117987.0


# Data Processing 

In [6]:
# nsrdb_na = df.drop(columns='Unnamed: 0')
nsrdb_na = df.copy()
#null_rows = nsrdb_na[nsrdb_na.isna().any(axis=1)]
#null_rows
nsrdb_na.isnull().sum()
#nsrdb_na.columns
#nsrdb_na['GHI'].isnull().values.sum()
nsrdb = nsrdb_na.dropna()
#print(nsrdb)
nsrdb.isnull().sum()

Unnamed: 0            0
Year                  0
Month                 0
Day                   0
Hour                  0
Minute                0
Temperature           0
Cloud_Type            0
Dew_Point             0
GHI                   0
Relative_Humidity     0
Solar_Zenith_Angle    0
Surface_Albedo        0
Pressure              0
Precipitable_Water    0
Wind_Direction        0
Wind_Speed            0
Location              0
dtype: int64

In [7]:
# Dropping Unnamed Column 
nsrdb = nsrdb.drop(['Unnamed: 0'], axis=1)
nsrdb

,Year,Month,Day,Hour,Minute,Temperature,Cloud_Type,Dew_Point,GHI,Relative_Humidity,Solar_Zenith_Angle,Surface_Albedo,Pressure,Precipitable_Water,Wind_Direction,Wind_Speed,Location
0,2018.0,1.0,1.0,0.0,0.0,11.2,4.0,10.2,0.0,93.63,165.41,0.100,1024.0,1.700,294.0,0.7,117987.0
1,2018.0,1.0,1.0,0.0,30.0,11.2,4.0,10.2,0.0,93.63,165.11,0.100,1024.0,1.700,307.0,0.7,117987.0
2,2018.0,1.0,1.0,1.0,0.0,11.1,4.0,10.1,0.0,93.38,162.29,0.100,1024.0,1.700,321.0,0.7,117987.0
3,2018.0,1.0,1.0,1.0,30.0,11.0,0.0,10.1,0.0,94.00,157.91,0.100,1024.0,1.800,332.0,0.7,117987.0
4,2018.0,1.0,1.0,2.0,0.0,10.9,0.0,9.9,0.0,93.32,152.72,0.100,1024.0,1.800,344.0,0.7,117987.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
946181,2013.0,12.0,31.0,21.0,30.0,6.0,1.0,0.0,0.0,74.05,142.58,0.113,1020.0,0.950,174.5,0.6,117546.0
946182,2013.0,12.0,31.0,22.0,0.0,6.0,1.0,0.0,0.0,74.40,148.35,0.113,1020.0,0.977,152.1,0.9,117546.0
946183,2013.0,12.0,31.0,22.0,30.0,5.0,0.0,0.0,0.0,79.80,153.89,0.113,1020.0,0.972,152.1,1.1,117546.0
946184,2013.0,12.0,31.0,23.0,0.0,4.0,0.0,0.0,0.0,80.32,158.95,0.113,1020.0,0.968,166.7,1.3,117546.0


In [8]:
nsrdb = nsrdb.loc[(nsrdb['Hour'].isin([7, 10, 13, 16, 19])) & (nsrdb['Minute'] == 0)]
nsrdb

,Year,Month,Day,Hour,Minute,Temperature,Cloud_Type,Dew_Point,GHI,Relative_Humidity,Solar_Zenith_Angle,Surface_Albedo,Pressure,Precipitable_Water,Wind_Direction,Wind_Speed,Location
14,2018.0,1.0,1.0,7.0,0.0,9.8,7.0,8.8,0.0,93.25,94.85,0.100,1025.0,1.800,325.0,0.7,117987.0
20,2018.0,1.0,1.0,10.0,0.0,14.0,7.0,9.5,290.0,74.48,67.81,0.100,1026.0,1.800,24.0,1.5,117987.0
26,2018.0,1.0,1.0,13.0,0.0,17.6,7.0,8.6,421.0,55.38,61.32,0.100,1024.0,1.800,15.0,0.9,117987.0
32,2018.0,1.0,1.0,16.0,0.0,15.4,7.0,9.2,114.0,66.62,80.55,0.100,1023.0,1.800,14.0,0.8,117987.0
38,2018.0,1.0,1.0,19.0,0.0,12.8,7.0,8.3,0.0,74.04,112.83,0.100,1024.0,1.700,72.0,1.0,117987.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
946152,2013.0,12.0,31.0,7.0,0.0,9.0,7.0,8.0,0.0,100.00,94.75,0.111,1020.0,1.233,174.6,0.4,117546.0
946158,2013.0,12.0,31.0,10.0,0.0,13.0,3.0,7.0,281.0,74.38,67.81,0.111,1020.0,1.403,23.4,0.7,117546.0
946164,2013.0,12.0,31.0,13.0,0.0,17.0,7.0,4.0,411.0,49.14,61.44,0.111,1020.0,1.334,33.3,1.1,117546.0
946170,2013.0,12.0,31.0,16.0,0.0,7.0,4.0,1.0,84.0,73.84,80.77,0.113,1020.0,0.999,343.0,0.8,117546.0


In [9]:
target = nsrdb['GHI'] # keeping only target(ghi) data here
input= nsrdb.drop(['GHI'], axis=1)
input

,Year,Month,Day,Hour,Minute,Temperature,Cloud_Type,Dew_Point,Relative_Humidity,Solar_Zenith_Angle,Surface_Albedo,Pressure,Precipitable_Water,Wind_Direction,Wind_Speed,Location
14,2018.0,1.0,1.0,7.0,0.0,9.8,7.0,8.8,93.25,94.85,0.100,1025.0,1.800,325.0,0.7,117987.0
20,2018.0,1.0,1.0,10.0,0.0,14.0,7.0,9.5,74.48,67.81,0.100,1026.0,1.800,24.0,1.5,117987.0
26,2018.0,1.0,1.0,13.0,0.0,17.6,7.0,8.6,55.38,61.32,0.100,1024.0,1.800,15.0,0.9,117987.0
32,2018.0,1.0,1.0,16.0,0.0,15.4,7.0,9.2,66.62,80.55,0.100,1023.0,1.800,14.0,0.8,117987.0
38,2018.0,1.0,1.0,19.0,0.0,12.8,7.0,8.3,74.04,112.83,0.100,1024.0,1.700,72.0,1.0,117987.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
946152,2013.0,12.0,31.0,7.0,0.0,9.0,7.0,8.0,100.00,94.75,0.111,1020.0,1.233,174.6,0.4,117546.0
946158,2013.0,12.0,31.0,10.0,0.0,13.0,3.0,7.0,74.38,67.81,0.111,1020.0,1.403,23.4,0.7,117546.0
946164,2013.0,12.0,31.0,13.0,0.0,17.0,7.0,4.0,49.14,61.44,0.111,1020.0,1.334,33.3,1.1,117546.0
946170,2013.0,12.0,31.0,16.0,0.0,7.0,4.0,1.0,73.84,80.77,0.113,1020.0,0.999,343.0,0.8,117546.0


# Feature Scaling

In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
input_normalized = sc.fit_transform(input)

# Split Data

In [11]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.3, random_state=42)

In [12]:
x_train

,Year,Month,Day,Hour,Minute,Temperature,Cloud_Type,Dew_Point,Relative_Humidity,Solar_Zenith_Angle,Surface_Albedo,Pressure,Precipitable_Water,Wind_Direction,Wind_Speed,Location
4334,2018.0,4.0,1.0,7.0,0.0,12.8,0.0,10.9,88.27,77.60,0.120,1018.0,1.300,252.0,2.5,117987.0
557524,2013.0,10.0,26.0,19.0,0.0,14.0,3.0,6.0,66.38,111.16,0.105,1000.0,1.085,313.2,1.1,116661.0
806338,2017.0,1.0,7.0,19.0,0.0,12.0,6.0,9.0,100.00,111.95,0.113,1010.0,3.485,107.1,1.3,117546.0
268226,2011.0,4.0,23.0,10.0,0.0,15.0,3.0,8.0,74.40,37.44,0.131,1010.0,2.406,180.7,2.0,117549.0
855008,2014.0,10.0,18.0,16.0,0.0,24.0,0.0,13.0,55.28,74.49,0.118,1010.0,2.705,280.7,2.2,117546.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60164,2011.0,6.0,8.0,7.0,0.0,13.0,0.0,10.0,93.97,66.28,0.118,1010.0,1.432,232.9,3.1,117987.0
526976,2017.0,1.0,28.0,10.0,0.0,10.0,8.0,4.0,70.85,64.46,0.101,1020.0,0.534,60.1,2.4,116661.0
737570,2014.0,2.0,4.0,7.0,0.0,6.0,7.0,4.0,97.20,92.36,0.117,1010.0,1.337,257.4,0.8,116662.0
8270,2018.0,6.0,22.0,7.0,0.0,17.5,0.0,12.5,72.61,66.52,0.100,1017.0,0.900,227.0,1.8,117987.0


In [13]:
print("x_train shape",x_train.shape)
print("x_test shape",x_test.shape)
print("y_train shape",y_train.shape)
print("y_text shape",y_test.shape)

x_train shape (68985, 16)
x_test shape (29565, 16)
y_train shape (68985,)
y_text shape (29565,)


#Training XGBoost on the Training set

In [14]:
from sklearn.datasets import make_classification
from xgboost import XGBClassifier
model = XGBClassifier(
    learning_rate=0.02,
    n_estimators=10,
    objective="binary:logistic",
    nthread=3,
    tree_method="gpu_hist"  # this enables GPU.
)



In [15]:
# fit the model
model.fit(x_train, y_train)

XGBClassifier(learning_rate=0.02, n_estimators=10, nthread=3,
              objective='multi:softprob', tree_method='gpu_hist')

In [16]:
y_pred=model.predict(x_test)
y_pred

array([  0., 703.,   0., ..., 408., 253.,   0.])

In [17]:
from sklearn.metrics import r2_score 
from sklearn import metrics
import math

R_square = r2_score(y_test, y_pred) 

print('Coefficient of Determination R2', R_square)
print("Model MAE", metrics.mean_absolute_error(y_test, y_pred))
print("Model MSE", metrics.mean_squared_error(y_test, y_pred))
print("Model RMSE",np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Coefficient of Determination R2 0.8659038117329848
Model MAE 53.03615761880602
Model MSE 13957.763504143413
Model RMSE 118.14297907257719


In [18]:
#from google.colab import drive
#drive.mount('/content/drive')

In [19]:
#model2.save('/content/drive/MyDrive/xgboost_final_16nov')

# GridSearchCV 

In [20]:
estimator = XGBClassifier(
    objective= 'binary:logistic',
    nthread=4,
    seed=42
)

In [23]:
parameters = {
    'max_depth': range (1, 2, 3),
    'n_estimators': range(10, 30, 100),
    'learning_rate': [0.1, 0.01, 0.001]
}

In [24]:
grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters,
    scoring = 'neg_mean_squared_error',
    n_jobs = None,
    cv = 3,
    verbose=True
)

In [25]:
grid_search.fit(x_train, y_train)
print("Best parameters:", grid_search.best_params_)
print("Lowest RMSE: ", (-grid_search.best_score_)**(1/2.0))

Fitting 3 folds for each of 3 candidates, totalling 9 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  UserWarning,


Best parameters: {'learning_rate': 0.1, 'max_depth': 1, 'n_estimators': 10}
Lowest RMSE:  128.04859928024524
